In [0]:
!pip install shap

In [ ]:
import pprint
import yaml
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import numpy as np
import pandas as pd

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from modelling.modelling import (
    build_linear_baseline,
    build_logistic_regression_model,
    build_random_forest_model,
    build_ensemble_random_forest,
    run_all_models,
    create_user_level_train_test_split,
    evaluate_on_holdout,
    EnsembleRandomForest,
)

from modelling.modelling_utils import (
    remove_collinear_features,
    calculate_vif_detailed,
    run_shap_analysis,
    analyze_feature_interactions,
    create_stakeholder_report,
    create_model_scorecard,
    analyze_logistic_calibration,
    generate_modelling_executive_summary,
    print_model_summary,
    get_significant_features,
    export_results,
    plot_shap_summary,
    plot_shap_bar,
    plot_shap_dependence,
)

# Load configs
with open('../configs/statistical_tests.yaml', 'r') as f:
    stats_config = yaml.safe_load(f)

with open('../configs/data_paths.yaml', 'r') as f:
    paths_config = yaml.safe_load(f)

pd.set_option('display.max_columns', None)
print("✓ Imports and configs loaded successfully")

In [0]:
# Loading user level dataframe
notebook_path = os.getcwd()
repo_root = os.path.abspath(os.path.join(notebook_path, ".."))
misc_dir = os.path.join(repo_root, "misc")

user_df_input_path = os.path.join(misc_dir,
                           os.path.basename(paths_config['output_files']['user_info_df_post_eda']))

test_results_df_input_path = os.path.join(misc_dir,
                           os.path.basename(paths_config['output_files']['test_results_df']))

user_info_df = pd.read_parquet(user_df_input_path)
test_results_df = pd.read_parquet(test_results_df_input_path)

user_info_df = user_info_df[~user_info_df['wonky_study_count'].isna()]
print(f"✓ Data loaded: {user_info_df.shape}")

In [0]:
user_info_df = user_info_df[~user_info_df['wonky_study_count'].isna()]
user_info_df.shape

#### Stage 1 - VIF & Correlation checks

In [0]:
test_results_df

In [0]:
significant_features = test_results_df['feature'].tolist()
significant_features = [item for item in significant_features if 'is_weekend' not in item]

print(f"Significant features from testing: {len(significant_features)}")
pprint.pprint(significant_features)

In [ ]:
# VIF Analysis - Identify multicollinearity risk for ALL features
vif_df = calculate_vif_detailed(
    df=user_info_df,
    feature_cols=significant_features,
    vif_threshold_high=10.0,
    vif_threshold_moderate=5.0,
)

print("\n" + "="*80)
print("VIF ANALYSIS - ALL FEATURES BY MULTICOLLINEARITY RISK")
print("="*80)
print("""
VIF Interpretation:
  - VIF = 1:    No correlation with other features
  - VIF < 5:    Low correlation (acceptable)
  - VIF 5-10:   Moderate correlation (monitor)
  - VIF > 10:   High correlation (consider removal)
  - VIF = inf:  Perfect multicollinearity (must remove)
""")
display(vif_df[['feature', 'VIF', 'vif_flag', 'recommendation']])

In [ ]:
# Visualize correlation between high-VIF (colinear) features
high_vif_features = vif_df[vif_df['VIF'] >= 5]['feature'].tolist()

if len(high_vif_features) >= 2:
    print(f"Plotting correlation matrix for {len(high_vif_features)} features with VIF >= 5")
    
    # Correlation heatmap of high-VIF features
    corr_matrix = user_info_df[high_vif_features].corr()

    fig = px.imshow(
        corr_matrix,
        text_auto='.2f',
        color_continuous_scale='RdBu_r',
        title='Correlation Matrix: High VIF Features (VIF >= 5)',
        aspect='auto',
    )
    fig.update_layout(
        width=800,
        height=800,
    )
    fig.show()
elif len(high_vif_features) == 1:
    print(f"Only 1 feature with high VIF: {high_vif_features[0]}")
    print("Cannot create correlation matrix with single feature")
else:
    print("No highly colinear features found (all VIF < 5) - no correlation chart needed")

In [ ]:
# OPTIONAL: Drop features with high multicollinearity
# Uncomment and modify the list below to exclude features before modelling

# features_to_drop = [
#     # Add feature names to drop here based on VIF analysis above, e.g.:
#     # 'is_weekend',
#     # 'is_business_hour',
# ]

# # Uncomment to apply the feature dropping:
# significant_features = [f for f in significant_features if f not in features_to_drop]
# print(f"Dropped {len(features_to_drop)} features: {features_to_drop}")
# print(f"Remaining features: {len(significant_features)}")

In [ ]:
# Run Random Forest + Logistic Regression (without Linear Regression due to multicollinearity)
# Clean features first
cleaned_features = remove_collinear_features(significant_features)

# Build Random Forest
rf_model, rf_importance, rf_cv_results = build_random_forest_model(
    df=user_info_df,
    feature_cols=cleaned_features,
    outcome_var="wonky_study_count",
    user_id_var="respondentPk",
    n_estimators=100,
)

# Build Logistic Regression with stronger Ridge (L2) regularization
# C=0.1 provides 10x stronger regularization than default (C=1.0)
# This helps stabilize coefficients when features are correlated
lr_model, lr_importance, lr_cv_results = build_logistic_regression_model(
    df=user_info_df,
    feature_cols=cleaned_features,
    outcome_var="wonky_study_count",
    user_id_var="respondentPk",
    regularization_C=0.1,  # Stronger Ridge penalty for multicollinearity
)

# Create results dict for compatibility with downstream cells
results = {
    'rf_model': rf_model,
    'rf_importance': rf_importance,
    'rf_cv_results': rf_cv_results,
    'lr_model': lr_model,
    'lr_importance': lr_importance,
    'lr_cv_results': lr_cv_results,
    'vif_data': vif_df,
    'feature_cols': cleaned_features,
}

# Create comparison table (RF + LR only)
comparison = rf_importance.merge(
    lr_importance[['feature', 'lr_coefficient', 'lr_odds_ratio', 'lr_p_value', 'lr_significant']],
    on='feature',
    how='outer'
)
comparison = comparison.merge(vif_df[['feature', 'VIF']], on='feature', how='left')

# Add rankings
comparison['rf_rank'] = comparison['rf_importance'].rank(ascending=False)
comparison['lr_rank'] = comparison['lr_coefficient'].abs().rank(ascending=False)
comparison['avg_rank'] = (comparison['rf_rank'] + comparison['lr_rank']) / 2

results['comparison'] = comparison.sort_values('avg_rank')

print(f"Models built with {len(cleaned_features)} features")
print(f"  - Random Forest CV R2: {rf_cv_results['cv_r2_mean']:.3f} (+/- {rf_cv_results['cv_r2_std']:.3f})")
print(f"  - Logistic Regression CV AUC: {lr_cv_results['cv_auc_mean']:.3f} (+/- {lr_cv_results['cv_auc_std']:.3f})")

In [0]:
results = run_shap_analysis(
    results=results,
    df=user_info_df,
    sample_size=1000,
)

#### Model Insights

In [0]:
if 'lr_importance' in results:
    print("\n" + "="*80)
    print("🎯 LOGISTIC REGRESSION - ODDS RATIOS")
    print("="*80)
    print("""
Odds Ratio Interpretation:
  • OR = 1.0:  No effect
  • OR = 1.5:  50% more likely to be wonky when feature=1
  • OR = 2.0:  2x more likely (100% increase)
  • OR = 0.5:  50% less likely (half the odds)
""")
    
    lr_df = results['lr_importance'].copy()
    
    # Add interpretation
    def interpret_or(x):
        if pd.isna(x) or x <= 0 or x > 100 or x < 0.01:
            return "⚠️ Extreme"
        if abs(x - 1) < 0.05:
            return "No effect"
        if x > 1:
            pct = (x - 1) * 100
            return f"↑ {pct:.0f}% more likely" if pct <= 100 else f"↑ {x:.1f}x"
        else:
            return f"↓ {(1-x)*100:.0f}% less likely"
    
    lr_df['interpretation'] = lr_df['lr_odds_ratio'].apply(interpret_or)
    
    # Filter valid odds ratios
    valid_lr = lr_df[(lr_df['lr_odds_ratio'] >= 0.01) & (lr_df['lr_odds_ratio'] <= 100)]
    
    display(valid_lr.head(20)[['feature', 'lr_coefficient', 'lr_odds_ratio', 
                               'interpretation', 'lr_p_value']])

In [0]:
print("\n" + "="*80)
print("🌲 RANDOM FOREST IMPORTANCE")
print("="*80)

rf_df = results['rf_importance'].copy()
display(rf_df.head(20)[['feature', 'rf_importance', 'rf_importance_pct']])

In [0]:
if 'shap_importance' in results:
    print("\n" + "="*80)
    print("🔮 SHAP FEATURE IMPORTANCE")
    print("="*80)
    print("""
SHAP shows HOW features affect predictions:
  • shap_importance: How much the feature matters
  • shap_mean: Direction (+ increases wonky, - decreases)
  • shap_direction: Plain English interpretation
""")
    
    shap_df = results['shap_importance'].copy()
    display(shap_df.head(20)[['feature', 'shap_importance', 'shap_mean', 'shap_direction']])

In [0]:
# Beeswarm plot - shows direction and distribution
plot_shap_summary(results, max_display=20)

In [0]:
plot_shap_bar(results, max_display=20)

In [0]:
# # to explore specific features using shap

# # How does is_saturday affect predictions?
# plot_shap_dependence(results, 'is_saturday')

# # With interaction coloring
# plot_shap_dependence(results, 'is_saturday', interaction_feature='is_early_morning')

In [0]:
print("\n" + "="*80)
print("🔍 MULTICOLLINEARITY CHECK (VIF)")
print("="*80)
print("""
VIF Interpretation:
  • VIF < 5:  ✓ No concern
  • VIF 5-10: ⚡ Moderate
  • VIF > 10: ⚠️ High - consider removing
""")

vif_df = results['vif_data'].copy()
vif_df['status'] = vif_df['VIF'].apply(
    lambda x: '✓ OK' if x < 5 else ('⚡ Moderate' if x < 10 else '⚠️ HIGH')
)

high_vif = vif_df[vif_df['VIF'] >= 5]
if len(high_vif) > 0:
    print(f"Features with VIF >= 5:")
    display(high_vif[['feature', 'VIF', 'status']])
else:
    print("✓ All features have VIF < 5")

In [0]:
print("\n" + "="*80)
print("📋 STAKEHOLDER REPORT")
print("="*80)

stakeholder_report = create_stakeholder_report(results)
display(stakeholder_report.head(20))

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)

# Top consensus features (based on RF + Logistic Regression)
print("\nTOP 5 CONSENSUS FEATURES (RF + LR):")
for i, (_, row) in enumerate(results['comparison'].head(5).iterrows(), 1):
    print(f"\n{i}. {row['feature']}")
    print(f"   RF importance: {row['rf_importance_pct']:.2f}%")
    if 'lr_odds_ratio' in row and pd.notna(row['lr_odds_ratio']):
        print(f"   Odds Ratio: {row['lr_odds_ratio']:.3f}")
    if 'lr_p_value' in row and pd.notna(row['lr_p_value']):
        print(f"   LR p-value: {row['lr_p_value']:.4e}")
    if 'shap_direction' in row and pd.notna(row.get('shap_direction')):
        print(f"   SHAP: {row['shap_direction']}")

# Features that increase vs decrease wonky
if 'shap_importance' in results:
    shap_df = results['shap_importance']
    
    increases = shap_df[shap_df['shap_mean'] > 0.001].head(5)
    decreases = shap_df[shap_df['shap_mean'] < -0.001].head(5)
    
    print("\nFEATURES THAT INCREASE WONKY:")
    for _, row in increases.iterrows():
        print(f"   {row['feature']}: +{row['shap_mean']:.4f}")
    
    print("\nFEATURES THAT DECREASE WONKY:")
    for _, row in decreases.iterrows():
        print(f"   {row['feature']}: {row['shap_mean']:.4f}")

In [ ]:
print("\n" + "="*80)
print("HIGH-CONFIDENCE FEATURES")
print("="*80)

# Features with high RF importance and significant in logistic regression
high_rf = set(results['rf_importance'][results['rf_importance']['rf_importance'] >= 0.01]['feature'].tolist())
sig_lr = set(results['lr_importance'][results['lr_importance']['lr_p_value'] < 0.05]['feature'].tolist())
high_confidence = high_rf.intersection(sig_lr)

print(f"\nFeatures with high RF importance + significant LR (p<0.05): {len(high_confidence)}")
for f in sorted(high_confidence):
    print(f"  - {f}")

#### Model Quality Assessment

Compare all models with a unified scorecard and calibration analysis.

In [ ]:
# Model Scorecard - compare all models at a glance
scorecard = create_model_scorecard(results)
display(scorecard)

In [ ]:
# Logistic Regression Calibration Analysis
# Check if predicted probabilities match actual frequencies
calibration_df, calibration_fig = analyze_logistic_calibration(
    results=results,
    df=user_info_df,
    n_bins=10,
)

if calibration_fig is not None:
    calibration_fig.show()
    
display(calibration_df)

#### Feature Interactions

Analyze how pairs of features work together to affect predictions.

In [ ]:
# Feature Interaction Analysis (can be slow - uses SHAP interaction values)
# Uncomment to run:
# interactions_df = analyze_feature_interactions(
#     results=results,
#     df=user_info_df,
#     top_n=10,
#     sample_size=500,
# )
# display(interactions_df)

#### Executive Summary

Comprehensive summary of modelling results for stakeholders.

In [ ]:
# Generate Executive Summary for Stakeholders
executive_summary = generate_modelling_executive_summary(
    results=results,
    outcome_description="wonkiness",
)

In [0]:
print("\n" + "="*80)
print("💾 EXPORT")
print("="*80)

# Export to CSV
export_df = export_results(results, 'feature_importance_results.csv')

# Also save as parquet for downstream use
results['comparison'].to_parquet('modelling_comparison.parquet', index=False)
print("✓ Results saved to modelling_comparison.parquet")

In [ ]:
# Interactive Plotly chart comparing RF importance vs LR Odds Ratios

comparison_df = results['comparison'].head(20).copy()

fig = go.Figure()

# RF importance
fig.add_trace(go.Bar(
    x=comparison_df['rf_importance_pct'],
    y=comparison_df['feature'],
    orientation='h',
    name='RF Importance (%)',
    marker=dict(color='#ff7f0e'),
))

# LR Odds Ratio (log scale for visualization, centered at 1)
# Convert OR to log scale so OR=0.5 and OR=2 are equidistant from OR=1
comparison_df['lr_or_log'] = np.log(comparison_df['lr_odds_ratio'].clip(lower=0.01, upper=100))
or_scale = comparison_df['rf_importance_pct'].max() / comparison_df['lr_or_log'].abs().max()

fig.add_trace(go.Bar(
    x=comparison_df['lr_or_log'] * or_scale,
    y=comparison_df['feature'],
    orientation='h',
    name='LR log(Odds Ratio) (scaled)',
    marker=dict(color='#2ca02c'),
))

fig.update_layout(
    title="Top 20 Features: RF Importance vs LR Odds Ratios",
    xaxis_title='Value (RF: %, LR: scaled log OR)',
    yaxis_title='Feature',
    barmode='group',
    yaxis=dict(autorange='reversed'),
    height=600,
    margin=dict(l=200),
)

fig.show()

In [ ]:
# The 'results' dictionary contains everything:
# - results['rf_model']: Random Forest model  
# - results['lr_model']: Logistic Regression model
# - results['rf_importance']: RF feature importance DataFrame
# - results['lr_importance']: LR coefficients and odds ratios DataFrame
# - results['vif_data']: VIF analysis DataFrame
# - results['comparison']: Unified comparison table (RF + LR)
# - results['shap_values']: SHAP values array
# - results['shap_explainer']: SHAP explainer object